In [1]:
require(stringr)
project_name = 'ML-MADNESS'
repo_path = file.path(substr(getwd(),0,gregexpr(pattern = project_name ,
                                                getwd())[[1]][1]-2),
                      project_name)
setwd(repo_path)

Loading required package: stringr



In [24]:
source(file.path(repo_path,'objects','helper_functions.R'))
p = scan(file.path(repo_path,'features/requirements.txt'),what="",sep="\n")
load_all_packages(p)
p = scan(file.path(repo_path,'sagemaker/requirements.txt'),what="",sep="\n")
load_all_packages(p)
slice = dplyr::slice
rename = dplyr::rename


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [3]:
# Sagemaker Settings
sagemaker <- import('sagemaker')
session <- sagemaker$Session()
bucket <- session$default_bucket()
role_arn <- sagemaker$get_execution_role()

In [4]:
source(file.path(repo_path,'features','api_calls.r'))

In [17]:
source(file.path(repo_path,'features','modeling.r'))
boxscores = load_box_score_data()

In [30]:
all_team_data = initial_cleaning(boxscores)

In [33]:
team_info = all_team_data %>%
  ungroup() %>% 
  select(team_id,team_display_name.x) %>% 
  group_by(team_id,team_display_name.x) %>% 
  summarize(APPEARANCES = n()) %>% 
  rename("team_name" = "team_display_name.x") %>% 
  arrange(desc(APPEARANCES))

team_data_refined = all_team_data %>% 
  ungroup() %>% 
  select(-c(team_display_name.x,team_display_name.y))

`summarise()` has grouped output by 'team_id'. You can override using the
`.groups` argument.


In [34]:
source(file.path(repo_path,'features','elo_helper.r'))

In [ ]:
df = calculate_relative_elo(team_data_refined = team_data_refined,
                            feature = "PTS.x",
                            season = 2022,
                            rename_f = "PTS_SCORED")

Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(columns)

  # Now:
  data %>% select(all_of(columns))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”
